In [30]:
import json

from decouple import config
import pandas as pd
import web3
from web3 import Web3

In [31]:
#connect to web3
w3 = Web3(Web3.HTTPProvider(config('MAINNET_ACCESS_URL'), request_kwargs={'timeout':60}))

In [32]:
#read abi
with open('../abi/abi.json') as f:
    abi = f.read()

#parse rinkeby data from http response
with open('../data/rinkeby.json') as f:
    data = json.loads(f.read())['result']

In [33]:
#contact contract
address = Web3.toChecksumAddress("0x88df592f8eb5d7bd38bfef7deb0fbc02cf3778a0")
tellor_contract = w3.eth.contract(address=address, abi=abi)

In [34]:
df = pd.DataFrame(data)
print(df.shape)
df.head()

(1000, 10)


,address,topics,data,blockNumber,timeStamp,gasPrice,gasUsed,logIndex,transactionHash,transactionIndex
0,0x88df592f8eb5d7bd38bfef7deb0fbc02cf3778a0,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x7ba54a,0x60304aca,0x3b9aca00,0x1ed07,0xb,0x6e036fa56f62ee9351656b2ed6c5f5c3742b00f49dba...,0x9
1,0x88df592f8eb5d7bd38bfef7deb0fbc02cf3778a0,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x7ba551,0x60304b33,0x3b9aca00,0x1b26f,0xd,0x3f665e769c0cfc0a487ad48fcd5e6c4f6067f933752e...,0xe
2,0x88df592f8eb5d7bd38bfef7deb0fbc02cf3778a0,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x7ba553,0x60304b51,0x3b9aca00,0x1b26f,0xa,0x860d9a2d8e7e0e23ca64053ff0186413f6e3438e3ee2...,0xc
3,0x88df592f8eb5d7bd38bfef7deb0fbc02cf3778a0,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x7ba554,0x60304b60,0x3b9aca00,0x1b26f,0xb,0xb8caf1907f6734b236447a87e59796c2e72575ce83b3...,0x8
4,0x88df592f8eb5d7bd38bfef7deb0fbc02cf3778a0,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x7ba555,0x60304b6f,0x3b9aca00,0x1b26f,0x5,0xedd5e608e0787e04df7cdeeda04c0f5282d60233a323...,0x7


In [36]:
transaction = w3.eth.getTransaction(df['transactionHash'][])
tellor_contract.decode_function_input(transaction.input)

(<Function submitMiningSolution(string,uint256[5],uint256[5])>,
 {'_nonce': 'anything will work',
  '_requestId': [5, 4, 3, 2, 1],
  '_value': [35546, 56113784471, 291620800, 55926910000, 1987860000]})

In [39]:
tx_receipt = w3.eth.getTransactionReceipt(df['transactionHash'][0])
logs = tellor_contract.events.Transfer().processReceipt(tx_receipt)

In [40]:
#Iterate through every row and parse to address, from address, and value
to_addresses = []
from_addresses = []
values = []
for i in range(df.shape[0]):
    tx_receipt = w3.eth.getTransactionReceipt(df['transactionHash'][0])
    logs = tellor_contract.events.Transfer().processReceipt(tx_receipt)
    _from = logs['args']['_from']
    _to = logs['args']['_to']


(AttributeDict({'args': AttributeDict({'_from': '0x0000000000000000000000000000000000000000',
   '_to': '0x0e93f23278161bCa6F22D6268ca44691042ed437',
   '_value': 1280300010000000129588}),
  'event': 'Transfer',
  'logIndex': 11,
  'transactionIndex': 9,
  'transactionHash': HexBytes('0x6e036fa56f62ee9351656b2ed6c5f5c3742b00f49dba0223acc538b8e44513fe'),
  'address': '0x88dF592F8eb5D7Bd38bFeF7dEb0fBc02cf3778a0',
  'blockHash': HexBytes('0x76562303db0b1892f64c8a34c196642c8273aca58815a68f638481f3c5d554a5'),
  'blockNumber': 8103242}),)